!pip install gym==0.26.2
!pip install gym-notices==0.0.8
!pip install gymnasium==0.29.1
!pip install gym-super-mario-bros==7.4.0
!pip install moviepy
!pip install opencv-python==4.9.0.80
!pip install ffmpeg-python==0.2.0

In [ ]:
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import gym.wrappers
import gym

import cv2
import os

import numpy as np

import glob

from IPython.display import HTML
from IPython.display import Video


In [ ]:
class FrameCaptureWrapper(gym.Wrapper):
    def __init__(self, env, video_filename="./mario-rl.mp4", fps=30.0):
        super(FrameCaptureWrapper, self).__init__(env)
        self.frames = []
        self.video_filename = video_filename
        self.fps = fps
        self.video = None

    def _open_video_writer(self):
        first_frame = self.env.render()
        height, width, _ = first_frame.shape
        self.video = cv2.VideoWriter(self.video_filename, cv2.VideoWriter_fourcc(*'mp4v'), self.fps, (width, height))
        #cv2.VideoWriter(output_name + '.avi', cv2.VideoWriter_fourcc('M','J','P','G'), fps, (source[0].shape[1], source[0].shape[0]), True)

    def _close_video_writer(self):
        if self.video is not None:
            self.video.release()

    def reset(self, **kwargs):
        self.frames = []
        self._close_video_writer()
        obs = self.env.reset(**kwargs)
        self._open_video_writer()
        return obs

    def step(self, action):
        if self.video is None:
            raise RuntimeError("Video writer is not initialized. Call reset before stepping.")
        obs, reward, terminated, truncated, info = self.env.step(action)
        self.video.write(cv2.cvtColor(self.env.render(), cv2.COLOR_RGB2BGR))
        return obs, reward, terminated, truncated, info

    def close(self):
        self._close_video_writer()
        super(FrameCaptureWrapper, self).close()

In [ ]:
#env = gym.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode="rgb_array")
env = gym_super_mario_bros.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode="rgb_array")

env = FrameCaptureWrapper(env, video_filename="./mario-rl.mp4", fps=30)
env.reset()

frames = []

for _ in range(1000):
    action = env.action_space.sample()  
    obs, reward, terminated, truncated, info = env.step(action)
    if terminated:
        obs = env.reset()

env.close()

frames = env.frames



In [ ]:
# Define the directory where your files are located
directory = "./"

# Define the file extension you're interested in
file_extension = "*.mp4"

# Use glob to search for files with the specified extension in the directory
file_list = glob.glob(directory + file_extension)

# Iterate through the list of files found
for file_path in file_list:
    # Do whatever you need to do with each file
    print("Found file:", file_path)


In [ ]:
file_path[0]

In [ ]:
#using embeded video player (jupyter or vscode)
#Video(filename=file_list[0],embed=True)

video_path = file_list[0]
# Embed the video using HTML5 video tag
HTML(f"""
<video width="640" height="480" controls>
  <source src="{video_path}" type="video/mp4">
</video>
""")